In [23]:
import random
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
import os

In [24]:
os.getcwd()
os.chdir(r'C:\\Users\\uzair\\Desktop\\Computer Vision\\cat_dog')
os.getcwd()

'C:\\Users\\uzair\\Desktop\\Computer Vision\\cat_dog'

In [25]:
if not os.path.exists(r'DataAug_Yolo\Labels'):
    os.makedirs(r'DataAug_Yolo\Labels')

In [26]:
with open('cat_dog_annotations.json') as f:
    data = json.load(f)

In [55]:
class_names = []

for j, (img_key, img_data) in enumerate(data['_via_img_metadata'].items()):
    filename = img_data['filename']
    regions = img_data['regions']
    
    image = cv2.imread(filename)
    shape = image.shape
    image_height, image_width = shape[0], shape[1]
    
    org_img_name = f'{j+1}_Original_image.jpeg'
    cv2.imwrite(f'DataAug_Yolo/{org_img_name}', image)
    
    # Generate unique label file name for each image
    label_file_name = f'{j+1}.txt'
    
    with open(os.path.join(r'DataAug_Yolo\Labels', label_file_name), 'w') as file:
        for region in regions:
            x, y = region['shape_attributes']['x'], region['shape_attributes']['y']
            w, h = region['shape_attributes']['width'], region['shape_attributes']['height']
            label = list(region['region_attributes']['class'].keys())[0]

            if label not in class_names:
                class_names.append(label)

            x_center = (x + w/2)/image_width
            y_center = (y + h/2)/image_height
            width = w/image_width
            height = h/image_height
            
            class_index = class_names.index(label)
            
            # Write label to the current image's label file
            file.write(f'{class_index} {x_center} {y_center} {width} {height}\n')
            
            for i in range(5):
                contrast = 1
                brightness = random.uniform(1.1, 1.5)
                bright_image = cv2.addWeighted(image, brightness, image, 0, contrast)      

                mean = 0
                sigma = random.randint(30, 50)
                gaussian_noise = np.random.normal(mean, sigma, shape)
                noised_image = cv2.add(image, gaussian_noise.astype(np.uint8))

                with open(os.path.join(r'DataAug_Yolo\Labels', f'{j+1}_brightness_{i+1}.txt'), 'w') as bright_file:
                    # Write label to the current image's brightness augmented label file
                    bright_file.write(f'{class_index} {x_center} {y_center} {width} {height}\n')

                bright_img_name = f'{j+1}_brightness_{i+1}.jpeg'
                cv2.imwrite(f'DataAug_Yolo/{bright_img_name}', bright_image)

                with open(os.path.join(r'DataAug_Yolo\Labels', f'{j+1}_noise_{i+1}.txt'), 'w') as noise_file:
                    # Write label to the current image's noise augmented label file
                    noise_file.write(f'{class_index} {x_center} {y_center} {width} {height}\n')

                noised_img_name = f'{j+1}_noise_{i+1}.jpeg'
                cv2.imwrite(f'DataAug_Yolo/{noised_img_name}', noised_image)
